# Azure AI Search custom vectorization sample
This sample demonstrates how to use Azure AI Search as a vector store by automatically chunking and generating embeddings using a custom embedding skill as part of the skillset pipeline in Azure AI Search. You can choose what embedding model works for your use case.
## Prerequisites
Install [azd](https://learn.microsoft.com/en-us/azure/developer/azure-developer-cli/overview) and run `azd up` to deploy the sample.
To run this sample query code, install the following packages. This sample currently uses version `11.4.0b11` of `azure-search-documents`

In [8]:
! pip install azure-search-documents==11.4.0b11 --quiet
! pip install python-dotenv azure-identity --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Load all environment variables from the azd deployment
import subprocess
from io import StringIO
from dotenv import load_dotenv
result = subprocess.run("azd env get-values", stdout=subprocess.PIPE)
load_dotenv(stream=StringIO(result.stdout.decode("utf-8")))

True

In [12]:
import os
search_url = f"https://{os.environ['AZURE_SEARCH_SERVICE']}.search.windows.net"

## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [29]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.identity import DefaultAzureCredential
# Pure Vector Search
query = "What's a performance review?"  
  
search_client = SearchClient(search_url, "custom-embedding-index", credential=DefaultAzureCredential())
vector_query = VectorizableTextQuery(text=query, k=1, fields="vector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  


parent_id: aHR0cHM6Ly9zdDN6emRiYm50cGxiMncuYmxvYi5jb3JlLndpbmRvd3MubmV0L2N1c3RvbS1lbWJlZGRpbmctc2FtcGxlLWRhdGEvZW1wbG95ZWVfaGFuZGJvb2sucGRm0
Score: 0.7918109
Content: for improvement. They will also provide you with 

an opportunity to discuss your goals and objectives for the upcoming year. 

 

Performance reviews are a two-way dialogue between managers and employees. We


## Perform a hybrid search

In [28]:
# Hybrid Search
query = "What's a performance review?"  
  
vector_query = VectorizableTextQuery(text=query, k=1, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  


parent_id: aHR0cHM6Ly9zdDN6emRiYm50cGxiMncuYmxvYi5jb3JlLndpbmRvd3MubmV0L2N1c3RvbS1lbWJlZGRpbmctc2FtcGxlLWRhdGEvZW1wbG95ZWVfaGFuZGJvb2sucGRm0
chunk_id: 9788cb956dcc_aHR0cHM6Ly9zdDN6emRiYm50cGxiMncuYmxvYi5jb3JlLndpbmRvd3MubmV0L2N1c3RvbS1lbWJlZGRpbmctc2FtcGxlLWRhdGEvZW1wbG95ZWVfaGFuZGJvb2sucGRm0_pages_15
Score: 0.029012346640229225
Content: for improvement. They will also provide you with 

an opportunity to discuss your goals and objectives for the upcoming year. 

 

Performance reviews are a two-way dialogue between managers and employees. We


## Perform a hybrid search + Semantic reranking

In [30]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "What's a performance review?"

vector_query = VectorizableTextQuery(text=query, k=2, fields="vector", exhaustive=True)

results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["parent_id", "chunk_id", "chunk"],
    query_type=QueryType.SEMANTIC,  semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=2
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Semantic Answer: <em>a rating of the 

employee’s performance, feedback, and goals and objectives for the upcoming year.</em> 

 

We understand that performance reviews can be a stressful process. We are committed to 

making sure that all employees feel supported and empowered during the process. We.
Semantic Answer Score: 0.92578125

parent_id: aHR0cHM6Ly9zdDN6emRiYm50cGxiMncuYmxvYi5jb3JlLndpbmRvd3MubmV0L2N1c3RvbS1lbWJlZGRpbmctc2FtcGxlLWRhdGEvZW1wbG95ZWVfaGFuZGJvb2sucGRm0
chunk_id: 9788cb956dcc_aHR0cHM6Ly9zdDN6emRiYm50cGxiMncuYmxvYi5jb3JlLndpbmRvd3MubmV0L2N1c3RvbS1lbWJlZGRpbmctc2FtcGxlLWRhdGEvZW1wbG95ZWVfaGFuZGJvb2sucGRm0_pages_14
Score: 0.01587301678955555
Content: conducted in an 

effective and efficient manner. 

 

Performance reviews are conducted annually and are an important part of your career 

development. During the review, your supervisor will discuss your performance over the 

past year and provide feedback on areas for improvement.
Caption: conducted in an 

effectiv